# งานที่ 2b: การสรุปข้อความนามธรรม

ในโน้ตบุ๊กนี้ คุณจะจัดการความท้าทายที่เกิดขึ้นในการสรุปเอกสารขนาดใหญ่ ซึ่งก็คือข้อความอินพุตอาจเกินความยาวบริบทของโมเดล มีการสร้างเอาต์พุตที่ไม่จริง หรือเกิดข้อผิดพลาดนอกหน่วยความจำ

เพื่อบรรเทาปัญหาเหล่านี้ โน้ตบุ๊กนี้จะแสดงให้เห็นถึงสถาปัตยกรรมโดยใช้การแบ่งพรอมต์คำสั่งและการเชื่อมต่อกับเฟรมเวิร์ก [LangChain] (https://python.langchain.com/docs/get_started/introduction.html) ซึ่งเป็นชุดเครื่องมือที่ช่วยให้แอปพลิเคชันใช้ประโยชน์จากโมเดลภาษาได้

คุณสำรวจแนวทางการแก้ไขสถานการณ์เมื่อเอกสารของผู้ใช้เกินขีด จำกัดโทเค็น การหั่นแยกเอกสารออกเป็นส่วนภายใต้เกณฑ์ความยาวของบริบทก่อนที่จะป้อนเอกสารไปยังโมเดลตามลำดับ ห่วงโซ่พรอมต์คำสั่งนี้จะแสดงให้เห็นเป็นชิ้นๆ โดยยังคงบริบทก่อนหน้านี้เอาไว้ คุณใช้วิธีการนี้เพื่อสรุปการถอดเสียงจากการโทร ถอดความประชุม หนังสือ บทความ โพสต์บล็อก และเนื้อหาที่เกี่ยวข้องอื่นๆ

## งานที่ 2b.1: การตั้งค่าสภาพแวดล้อม

ในงานนี้ ให้ตั้งค่าสภาพแวดล้อมของคุณ

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## งานที่ 2b.2: สรุปข้อความยาว 

### การกำหนดค่า LangChain ด้วย Boto3

ในงานนี้ คุณจำเป็นต้องระบุ LLM สำหรับคลาส LangChain Bedrock และสามารถส่งอาร์กิวเมนต์เพื่ออนุมานได้

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## งานที่ 2b.3: การโหลดไฟล์ข้อความที่มีโทเค็นจำนวนมาก

ในงานนี้ คุณสามารถค้นหาไฟล์ข้อความของ [จดหมายซีอีโอของ Amazon ถึงผู้ถือหุ้นในปี 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) ในไดเรกทอรีจดหมาย เซลล์ต่อไปนี้จะโหลดไฟล์ข้อความและนับจำนวนโทเค็น คุณจะเห็นคำเตือนที่ระบุจำนวนโทเค็นในไฟล์ข้อความที่เกินจำนวนโทเค็นสูงสุดสำหรับโมเดลนี้

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **หมายเหตุ: ** คุณสามารถเพิกเฉยต่อคำเตือนได้อย่างปลอดภัยและดำเนินการไปยังเซลล์ถัดไป

## งานที่ 2b.4: แบ่งข้อความยาวออกเป็นชิ้นๆ

ในงานนี้ คุณจะแบ่งข้อความออกเป็นชิ้นเล็กๆ เพราะข้อความนั้นยาวเกินพอกับพรอมต์คำสั่ง โดย `RecursiveCharacterTextSplitter` ใน LangChain รองรับการแบ่งข้อความยาวออกเป็นชิ้นเล็กๆ จนกว่าขนาดของแต่ละชิ้นจะเล็กกว่า `chunk_size` ข้อความจะถูกแยกด้วย `separators=["\n\n", "\n"]` `เป็นชิ้นๆ ซึ่งหลีกเลี่ยงการแบ่งแต่ละย่อหน้าออกเป็นหลายชิ้น

คุณสามารถรับบทสรุปสำหรับแต่ละส่วนแยกต่างหากได้สำหรับการใช้ 6,000 ตัวอักษรต่อชิ้น จำนวนโทเค็นหรือชิ้นส่วนคำในชิ้นส่วนขึ้นอยู่กับข้อความ

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## งานที่ 2b.5: สรุปชิ้นส่วนและรวมเข้าด้วยกัน

ในงานนี้ สมมติว่าจำนวนโทเค็นนั้นสอดคล้องกันในเอกสารอื่นๆ คุณควรจะสามารถทำได้ คุณสามารถใช้ `load_resumize_chain` ของ LangChain เพื่อสรุปข้อความ โดย `load_resumize_chain` มีสามวิธีในการสรุป: `stuff`, `map_reduce` และ `refine`

- `stuff`: ใส่ชิ้นส่วนทั้งหมดลงในพรอมต์คำสั่งเดียว ดังนั้น ข้อความจะถึงขีดจำกัดสูงสุดของโทเค็น
- `map_reduce`: สรุปแต่ละชิ้น รวมบทสรุป และสรุปรวมบทสรุป หากรวมบทสรุปมีขนาดใหญ่เกินไป จะทำให้เกิดข้อผิดพลาด
- `refine`: สรุปส่วนแรก แล้วสรุปส่วนที่สองด้วยสรุปแรก กระบวนการเดียวกันจะทำซ้ำจนกว่าชิ้นส่วนทั้งหมดจะถูกสรุป

ทั้ง map_reduce และ refinine จะเรียกใช้ LLM หลายครั้งและใช้เวลาในการรับสรุปสุดท้าย คุณสามารถลองใช้ map_reduce ได้ที่นี่

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **หมายเหตุ: ** ห่วงโซ่ด้านล่างอาจใช้เวลาสักครู่ในการทำงาน โดยขึ้นอยู่กับจำนวนเอกสาร โควต้าอัตราการร้องขอ Bedrock และการตั้งค่าการลองซ้ำที่กำหนดไว้

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

ตอนนี้คุณได้ทดลองใช้การแบ่งชิ้นคำสั่งและเชื่อมต่อกับเฟรมเวิร์ก LangChain เพื่อสรุปเอกสารขนาดใหญ่ในขณะที่บรรเทาปัญหาที่เกิดจากข้อความอินพุตยาวๆ

### ลองด้วยตัวเอง
- เปลี่ยนพรอมต์คำสั่งตามฐานการใช้งานเฉพาะของคุณและประเมินเอาต์พุตของโมเดลต่างๆ
- เล่นด้วยความยาวโทเค็นเพื่อทำความเข้าใจเวลาแฝงและการตอบกลับของบริการ
- ใช้หลักการวิศวกรรมการโต้ตอบที่แตกต่างกันเพื่อให้ได้เอาต์พุตที่ดีขึ้น

### เก็บงาน

คุณทำโน้ตบุ๊กนี้เสร็จแล้ว หากต้องการย้ายไปยังส่วนถัดไปของแล็บ ให้ทำดังนี้

- ปิดไฟล์โน้ตบุ๊กนี้และไปต่อยัง **งานที่ 3**